### Criar um dataframe para ler o arquivo no HDFS /user/thiago/data/juros_selic/juros_selic

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
juros_selic = spark.read.csv("/user/thiago/data/juros_selic/juros_selic", header=True, sep=";")

In [ ]:
juros_selic.show(3)

+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1,27|
|01/07/1986| 1,95|
|01/08/1986| 2,57|
+----------+-----+
only showing top 3 rows



### Agrupar todas as datas pelo ano em ordem decrescente e salvar a quantidade de meses ocorridos, o valor médio, mínimo e máximo do campo valor com a seguinte estrutura:

In [ ]:
# separando o ano da data, criando outro campo
juros_ano = juros_selic.withColumn("ano", split(col("data"), "/").getItem(2))
# transformando o dado do campo valor em float e substituindo no mesmo campo utilizando cast
juros_valor = juros_ano.withColumn("valor", regexp_replace(col("valor"), "\,", "\.").cast(FloatType()))

In [ ]:
juros_relatorio_ano = juros_valor.groupBy("ano").agg(count("ano").alias("Meses"),
format_number(avg("valor"), 2).alias("Valor Médio"),
min("valor").alias("Valor Mínimo"),
max("valor").alias("Valor Máximo")).sort(desc("ano"))

### Salvar no hdfs:///user/thiago/relatorioAnual com compressão zlib e formato orc

In [ ]:
juros_relatorio_ano.write.orc("/user/thiago/relatorio_anual", compression="zlib")

In [ ]:
!hdfs dfs -ls /user/thiago/relatorio_anual

Found 35 items
-rw-r--r--   2 root supergroup          0 2022-04-15 04:03 /user/thiago/relatorio_anual/_SUCCESS
-rw-r--r--   2 root supergroup        582 2022-04-15 04:03 /user/thiago/relatorio_anual/part-00000-d121e533-9003-4cac-87b5-78884404607c-c000.zlib.orc
-rw-r--r--   2 root supergroup        582 2022-04-15 04:03 /user/thiago/relatorio_anual/part-00001-d121e533-9003-4cac-87b5-78884404607c-c000.zlib.orc
-rw-r--r--   2 root supergroup        585 2022-04-15 04:03 /user/thiago/relatorio_anual/part-00002-d121e533-9003-4cac-87b5-78884404607c-c000.zlib.orc
-rw-r--r--   2 root supergroup        572 2022-04-15 04:03 /user/thiago/relatorio_anual/part-00003-d121e533-9003-4cac-87b5-78884404607c-c000.zlib.orc
-rw-r--r--   2 root supergroup        583 2022-04-15 04:03 /user/thiago/relatorio_anual/part-00004-d121e533-9003-4cac-87b5-78884404607c-c000.zlib.orc
-rw-r--r--   2 root supergroup        582 2022-04-15 04:03 /user/thiago/relatorio_anual/part-00005-d121e533-9003-4cac-87b5-78884404607c-c0